In [1]:
import argparse
import os
import random
import time
from functools import partial

import numpy as np
import paddle
import pandas as pd
from data import convert_pairwise_example
from model import PairwiseMatching
from tqdm import tqdm

from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoModel, AutoTokenizer, LinearDecayWithWarmup

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
# 下面加载训练集和验证集

In [2]:
def read_train(data_path): 
    with open(data_path, 'r', encoding='utf-8') as f: 
        flag = 0
        for line in f: 
            if flag != 0: 
                data = line.rstrip().split("\t") 
                if len(data) != 3: 
                    continue
                yield {'query': data[0], 'title': data[1], 'neg_title': data[2]} 
                                                                                   
            flag = 1

In [3]:
def read_dev(data_path): 
    with open(data_path, 'r', encoding='utf-8') as f: 
        flag = 0
        for line in f: 
            if flag != 0: 
                data = line.rstrip().split("\t") 
                if len(data) != 3: 
                    continue
                yield {'query': data[0], 'title': data[1], 'label': data[2]} 
                                                                               
            flag = 1

In [4]:
train_file = "rank_dataset/train_pairwise.csv"
train_ds = load_dataset(read_train, data_path=train_file, lazy=False) 

dev_file = "rank_dataset/dev_pairwise.csv"
dev_ds = load_dataset(read_dev, data_path=dev_file, lazy=False) 

In [5]:
# 输出训练数据集的前3条数据看一下
for i in range(3):
    print(train_ds[i])

{'query': '英语委婉语引起的跨文化交际障碍', 'title': '英语委婉语引起的跨文化交际障碍及其翻译策略研究英语委婉语,跨文化交际障碍,翻译策略', 'neg_title': '委婉语在英语和汉语中的文化差异委婉语,文化,跨文化交际'}
{'query': '范迪慧 嘉兴市中医院', 'title': '滋阴疏肝汤联合八穴隔姜灸治疗肾虚肝郁型卵巢功能低下的临床疗效滋阴疏肝汤,八穴隔姜灸,肾虚肝郁型卵巢功能低下,性脉甾类激素,妊娠', 'neg_title': '温针灸、中药薰蒸在半月板损伤术后康复中的疗效分析膝损伤,半月板,胫骨,中医康复,温针疗法,薰洗'}
{'query': '灰色关联分析', 'title': '灰色关联分析评价不同产地金果榄质量金果榄;灰色关联分析法;主成分分析法;盐酸巴马汀;盐酸药根碱', 'neg_title': '江西省某三级甲等医院2型糖尿病患者次均住院费用新灰色关联分析2型糖尿病,次均住院费用,新灰色关联分析,结构变动度'}


In [6]:
# 输出验证数据集的前3条数据看一下
for i in range(3):
    print(dev_ds[i])

{'query': '作者单位:南州中学', 'title': '浅谈初中教学管理如何体现人文关怀初中教育,教学管理,人文关怀', 'label': '1'}
{'query': '作者单位:南州中学', 'title': '高中美术课堂教学中藏区本土民间艺术的融入路径藏区,传统民间艺术,美术课堂', 'label': '0'}
{'query': '作者单位:南州中学', 'title': '列宁关于资产阶级民主革命向 社会主义革命过渡的理论列宁,直接过渡,间接过渡,资产阶级民主革命,社会主义革命', 'label': '0'}


In [7]:
model_name = "ernie-3.0-medium-zh"

tokenizer = AutoTokenizer.from_pretrained(model_name) 

[2025-02-06 23:22:46,906] [    INFO] - We are using (<class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'>, False) to load 'ernie-3.0-medium-zh'.
[2025-02-06 23:22:46,907] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh_vocab.txt and saved to C:\Users\LJX\.paddlenlp\models\ernie-3.0-medium-zh
[2025-02-06 23:22:47,135] [    INFO] - Downloading ernie_3.0_medium_zh_vocab.txt from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh_vocab.txt
100%|██████████| 182k/182k [00:00<00:00, 376kB/s] 
[2025-02-06 23:22:47,896] [    INFO] - tokenizer config file saved in C:\Users\LJX\.paddlenlp\models\ernie-3.0-medium-zh\tokenizer_config.json
[2025-02-06 23:22:47,897] [    INFO] - Special tokens file saved in C:\Users\LJX\.paddlenlp\models\ernie-3.0-medium-zh\special_tokens_map.json


In [8]:
trans_func_train = partial(convert_pairwise_example, tokenizer=tokenizer, max_seq_length=128, phase="train")

trans_func_eval = partial(convert_pairwise_example, tokenizer=tokenizer, max_seq_length=128, phase="eval")

In [9]:
#演示
example = train_ds[0]
print(example)

query, pos_title, neg_title = example["query"], example["title"], example["neg_title"]
print(query)
print(pos_title)
print(neg_title)
print('---------------------------------------------')

pos_inputs = tokenizer(text=query, text_pair=pos_title, max_seq_len=128) 
neg_inputs = tokenizer(text=query, text_pair=neg_title, max_seq_len=128) 

print(pos_inputs)
print(neg_inputs)
print('---------------------------------------------')

pos_input_ids = pos_inputs["input_ids"] 
pos_token_type_ids = pos_inputs["token_type_ids"] 

print(pos_input_ids)
print(pos_token_type_ids)
print('---------------------------------------------')

neg_input_ids = neg_inputs["input_ids"] 
neg_token_type_ids = neg_inputs["token_type_ids"] 
print(neg_input_ids)
print(neg_token_type_ids)
print('---------------------------------------------')

result = [pos_input_ids, pos_token_type_ids, neg_input_ids, neg_token_type_ids]
print(result)

{'query': '英语委婉语引起的跨文化交际障碍', 'title': '英语委婉语引起的跨文化交际障碍及其翻译策略研究英语委婉语,跨文化交际障碍,翻译策略', 'neg_title': '委婉语在英语和汉语中的文化差异委婉语,文化,跨文化交际'}
英语委婉语引起的跨文化交际障碍
英语委婉语引起的跨文化交际障碍及其翻译策略研究英语委婉语,跨文化交际障碍,翻译策略
委婉语在英语和汉语中的文化差异委婉语,文化,跨文化交际
---------------------------------------------
{'input_ids': [1, 514, 405, 298, 2681, 405, 477, 200, 5, 1465, 68, 73, 276, 430, 843, 1767, 2, 514, 405, 298, 2681, 405, 477, 200, 5, 1465, 68, 73, 276, 430, 843, 1767, 107, 63, 1197, 1285, 584, 750, 166, 229, 514, 405, 298, 2681, 405, 30, 1465, 68, 73, 276, 430, 843, 1767, 30, 1197, 1285, 584, 750, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [1, 514, 405, 298, 2681, 405, 477, 200, 5, 1465, 68, 73, 276, 430, 843, 1767, 2, 298, 2681, 405, 11, 514, 405, 14, 657, 405, 12, 5, 68, 73, 859, 712, 298, 2681, 405, 30, 68, 73, 30, 1465, 68, 73, 276, 430, 2], 'token_type_ids

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2293: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1865: UserWarning: Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  warnings.warn(


In [10]:
def batchify_fn_train(samples): 
    fn = Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype="int64"),  
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype="int64"),  
    )

    processed_samples = fn(samples) 

    result = []
    for data in processed_samples:
        result.append(data) 

    return result

In [11]:
def batchify_fn_eval(samples): 
    fn = Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype="int64"),  
        Stack(dtype="int64"), 
    )

    processed_samples = fn(samples) 

    result = []
    for data in processed_samples:
        result.append(data) 

    return result

In [12]:
batch_sampler_train = paddle.io.DistributedBatchSampler(train_ds, batch_size=32, shuffle=True)

train_data_loader = paddle.io.DataLoader(dataset=train_ds.map(trans_func_train), batch_sampler=batch_sampler_train, collate_fn=batchify_fn_train, return_list=True)

In [13]:
batch_sampler_dev = paddle.io.BatchSampler(dev_ds, batch_size=32, shuffle=False)

dev_data_loader = paddle.io.DataLoader(dataset=dev_ds.map(trans_func_eval), batch_sampler=batch_sampler_dev, collate_fn=batchify_fn_eval, return_list=True)

In [15]:
# 下面搭建模型，并开始训练

In [14]:
pretrained_model = AutoModel.from_pretrained(model_name) 

[2025-02-06 23:28:09,072] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieModel'> to load 'ernie-3.0-medium-zh'.
[2025-02-06 23:28:09,075] [    INFO] - Configuration saved in C:\Users\LJX\.paddlenlp\models\ernie-3.0-medium-zh\config.json
[2025-02-06 23:28:09,301] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams
[2025-02-06 23:28:09,304] [    INFO] - Downloading ernie_3.0_medium_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams
100%|██████████| 313M/313M [00:26<00:00, 12.6MB/s] 
[2025-02-06 23:28:35,847] [    INFO] - Loading weights file model_state.pdparams from cache at C:\Users\LJX\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2025-02-06 23:28:36,067] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-02-06 23:28:37,078] [ WARNING] - Some weights of the model checkpoint at ernie-3.0-medium-zh w

In [15]:
model = PairwiseMatching(pretrained_model, margin=0.1) 

In [16]:
epochs = 3 

num_training_steps = len(train_data_loader) * epochs 

lr_scheduler = LinearDecayWithWarmup(2E-5, num_training_steps, 0.0)

In [17]:
decay_params = [
    p.name for n, p in model.named_parameters() 
    if not any(nd in n for nd in ["bias", "norm"])
]


In [18]:
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=0.0,
    apply_decay_param_fun=lambda x: x in decay_params)

In [19]:
metric = paddle.metric.Auc() 

In [20]:
@paddle.no_grad() 
def evaluate(model, metric, data_loader): 
    model.eval() 
    metric.reset()
 
    for idx, batch in enumerate(data_loader): 
        input_ids, token_type_ids, labels = batch
        pos_probs = model.predict(input_ids=input_ids, token_type_ids=token_type_ids) 
        neg_probs = 1.0 - pos_probs 
        preds = np.concatenate((neg_probs, pos_probs), axis=1) 

        metric.update(preds=preds, labels=labels) 
        auc = metric.accumulate() 

    print("phase: dev, auc: {:.3}".format(auc)) 
    metric.reset()
    model.train()

In [21]:
# 下面正式开始训练模型

save_dir="model_param"
global_step = 0
tic_train = time.time()

for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1): 
        pos_input_ids, pos_token_type_ids, neg_input_ids, neg_token_type_ids = batch

        loss = model(
            pos_input_ids=pos_input_ids,
            neg_input_ids=neg_input_ids,
            pos_token_type_ids=pos_token_type_ids,
            neg_token_type_ids=neg_token_type_ids,
        ) 

        global_step += 1

        if global_step % 10 == 0: 
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, 10 / (time.time() - tic_train)))
            tic_train = time.time()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()

        if global_step % 100 == 0:
            evaluate(model, metric, dev_data_loader)
            
            save_path = os.path.join(save_dir, "model_%d" % global_step) 
            if not os.path.exists(save_path):
                os.makedirs(save_path)

            save_param_path = os.path.join(save_path, "model_state.pdparams") 
            paddle.save(model.state_dict(), save_param_path)
            tokenizer.save_pretrained(save_path) 

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2293: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(


global step 10, epoch: 1, batch: 10, loss: 0.09501, speed: 0.58 step/s
global step 20, epoch: 1, batch: 20, loss: 0.09584, speed: 0.60 step/s
global step 30, epoch: 1, batch: 30, loss: 0.09322, speed: 0.54 step/s
global step 40, epoch: 1, batch: 40, loss: 0.07434, speed: 0.68 step/s
global step 50, epoch: 1, batch: 50, loss: 0.09631, speed: 0.64 step/s
global step 60, epoch: 1, batch: 60, loss: 0.08708, speed: 0.59 step/s
global step 70, epoch: 1, batch: 70, loss: 0.09902, speed: 0.58 step/s
global step 80, epoch: 1, batch: 80, loss: 0.06268, speed: 0.59 step/s
global step 90, epoch: 1, batch: 90, loss: 0.07621, speed: 0.57 step/s


c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2989: UserWarning: Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  warnings.warn(


global step 100, epoch: 1, batch: 100, loss: 0.08093, speed: 0.60 step/s
phase: dev, auc: 0.769


[2025-02-07 00:30:47,193] [    INFO] - tokenizer config file saved in model_param\model_100\tokenizer_config.json
[2025-02-07 00:30:47,195] [    INFO] - Special tokens file saved in model_param\model_100\special_tokens_map.json


...

[2025-02-08 00:08:31,636] [    INFO] - tokenizer config file saved in model_param\model_10800\tokenizer_config.json
[2025-02-08 00:08:31,638] [    INFO] - Special tokens file saved in model_param\model_10800\special_tokens_map.json


global step 10810, epoch: 1, batch: 10810, loss: 0.07401, speed: 0.02 step/s
global step 10820, epoch: 1, batch: 10820, loss: 0.09065, speed: 0.54 step/s
global step 10830, epoch: 1, batch: 10830, loss: 0.07131, speed: 0.66 step/s
global step 10840, epoch: 1, batch: 10840, loss: 0.05337, speed: 0.50 step/s
global step 10850, epoch: 1, batch: 10850, loss: 0.07657, speed: 0.44 step/s
global step 10860, epoch: 1, batch: 10860, loss: 0.06588, speed: 0.52 step/s
global step 10870, epoch: 1, batch: 10870, loss: 0.06121, speed: 0.55 step/s
global step 10880, epoch: 1, batch: 10880, loss: 0.07449, speed: 0.60 step/s
global step 10890, epoch: 1, batch: 10890, loss: 0.10121, speed: 0.49 step/s
global step 10900, epoch: 1, batch: 10900, loss: 0.09062, speed: 0.45 step/s
phase: dev, auc: 0.784


[2025-02-08 00:22:26,046] [    INFO] - tokenizer config file saved in model_param\model_10900\tokenizer_config.json
[2025-02-08 00:22:26,048] [    INFO] - Special tokens file saved in model_param\model_10900\special_tokens_map.json


global step 10910, epoch: 1, batch: 10910, loss: 0.05325, speed: 0.02 step/s


KeyboardInterrupt: 

In [22]:
# 下面是把保存好的模型参数文件加载到一个新的模型中，并进行性能测试的过程

model_2 = PairwiseMatching(pretrained_model, margin=0.1)

params_path = "model_param/model_400/model_state.pdparams"

state_dict = paddle.load(params_path)
model_2.set_dict(state_dict)

evaluate(model_2, metric, dev_data_loader)

KeyboardInterrupt: 

In [ ]:
# 下面是把保存好的模型参数文件加载到一个新的模型中，并进行性能测试的过程

model_2 = PairwiseMatching(pretrained_model, margin=0.1)

params_path = "model_param/model_10700/model_state.pdparams"

state_dict = paddle.load(params_path)
model_2.set_dict(state_dict)

evaluate(model_2, metric, dev_data_loader)